# CharacterList要有資料
## 如CharacterList無資料請先執行GetCharacterList取得角色清單

In [432]:
import configparser
from Tool import CrawlerTool
from Tool import JSONTool

In [433]:
characterUpgradeItemList = []

# 讀取config設定
config = configparser.ConfigParser()
config.read('config.ini')

characterListFilePath = config['JSONPath']['CharacterList']
characterUpgradeItemListPath = config['JSONPath']['CharacterUpgradeItemList']

In [434]:
def GetUpgradeItem(area, itemType, characterName, ratio = 1):
    itemList = []
    item = None

    for span in area.find_all("span"):                                                              # 找到此區域所有span區塊
        if span.find("span", class_ = "card-body") and "card-list-container" not in span["class"]:  # 如果這個span class是card-body
            itemTag = span.find("a", title = True)                                                  # 從span區快中找出第一個 <a> 標籤，且該標籤有title屬性的元素
            item = itemTag["title"] if itemTag else None                                            # 取得a的title

            valueTag = span.find("span", class_ = "card-text card-font")                            # 找到此span區塊裡class = card-text card-font的子span
            value = valueTag.get_text(strip = True) if valueTag else None                           # 取得子span值
            value = int(value.replace(",", ""))
            
            if value is not None:
                itemList.append({"Character": characterName, "ItemType": itemType, "Item": item, "Value": value * ratio})

    return itemList

In [435]:
characterList = JSONTool.Read(characterListFilePath)
#character = characterList[0]
for character in characterList:
    #取得角色頁面資訊
    soup = CrawlerTool.GetScop(character["Link"])

    itemList = []

    area = None

    #===============================等級提升素材===============================
    for span in soup.find_all("span"):
        # 確保沒有 class 或 id 屬性
        if not span.has_attr("class") and not span.has_attr("id"):
            # 檢查文字是否包含目標內容
            if "(0✦ → 6✦)" in span.get_text(strip=True):
                area = span
                break

    itemList += GetUpgradeItem(area, "等級提升素材", character["Name"])
    #===============================等級提升素材===============================

    #===============================技能提升素材===============================
    for div in soup.find_all("div"):
        # 確保沒有 class 或 id 屬性
        if not div.has_attr("class") and not div.has_attr("id"):
            # 檢查文字是否包含目標內容
            if "(1 → 10 for One Main Forte)" in div.get_text(strip=True):
                area = div
                break

    itemList += GetUpgradeItem(area, "技能提升素材", character["Name"], 5)
    #===============================技能提升素材===============================

    #===============================固有技能提升素材===========================
    for div in soup.find_all("div"):
        # 確保沒有 class 或 id 屬性
        if not div.has_attr("class") and not div.has_attr("id"):
            # 檢查文字是否包含目標內容
            if "(1 → 2 for All Inherent Skill)" in div.get_text(strip=True):
                area = div
                break

    itemList += GetUpgradeItem(area, "固有技能提升素材", character["Name"])
    #===============================固有技能提升素材===========================

    #===============================屬性提升素材===============================
    for div in soup.find_all("div"):
        # 確保沒有 class 或 id 屬性
        if not div.has_attr("class") and not div.has_attr("id"):
            # 檢查文字是否包含目標內容
            if "(1 → 2 for One Strand of Stat Bonus)" in div.get_text(strip=True):
                area = div
                break

    itemList += GetUpgradeItem(area, "屬性提升素材", character["Name"], 4)
    #===============================屬性提升素材===============================

    characterUpgradeItemList.append(itemList)




In [436]:
#print(characterUpgradeItemList)
JSONTool.Write(characterUpgradeItemListPath, characterUpgradeItemList)